# Assignment

## Importing Libraries

In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Currently using {device}')

Currently using cpu


## Loading the Datasets

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

cifar_dataset_train = datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)

cifar_dataset_test = datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
classes = ()

## Creating the training dataset

In [4]:
from torch.utils.data import DataLoader
batch_size = 100

train_iter = DataLoader(cifar_dataset_train,batch_size,shuffle=True,num_workers=2)

In [5]:
X,y = next(iter(train_iter))

## Creating the test dataset

In [6]:
test_iter = DataLoader(cifar_dataset_test,batch_size,shuffle=False,num_workers=2)

In [7]:
X_test,y_test = next(iter(test_iter))

## Analysing the CIFAR-10 data

In [8]:
print(f'The dimensions of each input image is {X.size()} The number of input channels is {X.size()[1]}')
print(f'The output labels are as follows {y}')

The dimensions of each input image is torch.Size([100, 3, 32, 32]) The number of input channels is 3
The output labels are as follows tensor([7, 7, 1, 3, 9, 8, 4, 0, 6, 1, 0, 7, 5, 8, 7, 9, 7, 9, 4, 1, 0, 0, 8, 8,
        0, 6, 2, 4, 4, 6, 3, 3, 8, 8, 5, 2, 7, 6, 5, 2, 7, 7, 3, 0, 6, 0, 7, 2,
        7, 0, 4, 3, 7, 1, 8, 1, 4, 9, 9, 3, 4, 3, 1, 1, 9, 5, 3, 6, 1, 9, 5, 3,
        1, 5, 6, 7, 4, 1, 2, 0, 7, 6, 3, 8, 2, 3, 0, 3, 3, 9, 1, 2, 7, 3, 9, 6,
        4, 6, 8, 8])


## Example image

## CNN Architecture

In [81]:
class Block(torch.nn.Module):
    def __init__(self,input_channels,output_channels,features,out_features):
        super(Block,self).__init__()

        self.block = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=3,stride=2,padding=1),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=features,out_features=out_features),
            torch.nn.ReLU(),
            torch.nn.Dropout(0)
        )

    def forward(self,x):
        linear = x

        block = self.block(x)
        linear = self.linear(torch.flatten(x))

        block_out = torch.matmul(linear,block)

        x = torch.nn.functional.relu(block_out)

        return x

In [100]:
class CNN(torch.nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()

        self.stem = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),          
            torch.nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
        )

        self.block1 = Block(input_channels=3,output_channels=64,features=307200,out_features=16)
        self.block2 = Block(input_channels=64,output_channels=128,features=102400,out_features=8)

        self.last = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((4,4)),
                                        torch.nn.Flatten(),
                                        torch.nn.Linear(in_features=16,out_features=16),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(0),
                                        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=16,out_features=num_classes),
            torch.nn.ReLU(),
            torch.nn.Dropout(0),
            torch.nn.Softmax(dim=1)
        )

    def forward(self,x):

        output = self.stem(x)

        output = self.block1(output)

        output = self.block2(output.view(100,64,16,1))

        output = self.last(output)

        output = self.linear(output)

        return output

In [101]:
model = CNN(num_classes=10).to(device)
print(model)

CNN(
  (stem): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=3, stride=1, padding=1)
  )
  (block1): Block(
    (block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=307200, out_features=16, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
  )
  (block2): Block(
    (block): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=102400, out_features=8, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
  )
  (last): Sequential(
    (0): Adap

In [102]:
model(X).size()

torch.Size([100, 10])

## Create Loss and Optimization

In [103]:
loss = torch.nn.CrossEntropyLoss()

# Create the optimizer term
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005,weight_decay=0.005,momentum=0.9)

## Calculate Loss, and Validation Accuracy

In [104]:
def train_test_model(model,train_iter,test_iter,epochs,optimizer,loss):
    for epoch in range(epochs): # Initialize the epochs

        l_score = []

        for i,(X,y) in enumerate(train_iter):

            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            #model.train()
            y_hat = model(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {l.item():.4f}')

        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_iter:
                X_test,y_test = data
                X_test = X_test.to(device)
                y_test = y_test.to(device)

                y_hat = model(X_test)

                _, predicted = torch.max(y_hat.data,1)
                total += y_test.size(0)
                correct += (predicted == y_test).sum().item()

            print(f'Accuracy of the network on the test images is : {100 * correct // total} percent')

    return

In [105]:
epochs = 5
train_test_model(model,train_iter,test_iter,epochs,optimizer,loss)

Epoch [1/5], Loss: 2.2261
Accuracy of the network on the test images is : 21 percent
Epoch [2/5], Loss: 2.2323
Accuracy of the network on the test images is : 23 percent
Epoch [3/5], Loss: 2.2701
Accuracy of the network on the test images is : 24 percent
Epoch [4/5], Loss: 2.1824
Accuracy of the network on the test images is : 26 percent
Epoch [5/5], Loss: 2.2342
Accuracy of the network on the test images is : 27 percent
